**ETAPA** 1
===
Avaliar a qualidade das respostas, independente do gabarito, pautadas na ocorrência de:
1. erros de grafia;
2. erros gramaticais;  

In [9]:
respostas = ['Tais comandos devem ser colocados dentro de um bloco try. Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.','Tais comandos devem ser colocados dentro de um bloco try. Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.','Tais comandos devem ser colocados dentro de um bloco try. Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.','Tais comandos devem ser colocados dentro de um bloco try. Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.']

import re
import SpellChecker as sc
from nltk.corpus import wordnet

respostasSemPontuacao=[]
for r in respostas:
    respostasSemPontuacao.append(re.sub('[,.]', '', r))
    
for r in respostasSemPontuacao:
	words = r.lower().split()

# Verifica o número de erros entre os termos e a correção
errors = 0
doc_size = len(words)
for w in words:
    if not wordnet.synsets(w): #se o termo não estiver em inglês
        if w != sc.correction(w):
            print(w," ~> ",sc.correction(w))
            errors+=1
err_per=(errors/doc_size)*100
print(errors," erro(s)\n(%%): %.2f" %err_per)

comandos  ~>  comando
bloco  ~>  floco
bloco  ~>  floco
comandos  ~>  comando
bloco  ~>  floco
controle  ~>  console
bloco  ~>  floco
parâmetro  ~>  barômetro
8  erro(s)
(%): 16.00


ANÁLISE GRAMATICAL
===
> A Análise gramatical é realizada sobre o conjunto de respostas original, ou seja, considera-se o texto na íntegra, com pontuações, etc.

> Uso do Cogroo (https://github.com/gpassero/cogroo4py) para análise gramatical

* Deve-se executar o arquivo cogroo4py.jar no console

> ``` $ java -jar cogroo4py.jar & ```


In [15]:
from cogroo_interface import Cogroo

cogroo = Cogroo.Instance()
# Avalia cada sentença de cada resposta, apresentando o(s) erro(s) da determinada 
#sentença
for r in respostas:
    sdoc = cogroo.grammar_check(r)
    for s in sdoc.sentences:
        str_sentences = str(s)
        d = cogroo.grammar_check(str_sentences)
        err = d.mistakes
        print("\n\n",str_sentences," ~ ",err, "\n\n")
    print("\n\nErros: ", len(sdoc.mistakes))





 Tais comandos devem ser colocados dentro de um bloco try.  ~  [] 




 Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.  ~  [[xml:105] Os artigos concordam com o substantivo a que se referem., [xml:114] Os determinantes concordam com o substantivo a que se referem.] 




Erros:  2


 Tais comandos devem ser colocados dentro de um bloco try.  ~  [] 




 Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.  ~  [[xml:105] Os artigos concordam com o substantivo a que se referem., [xml:114] Os determinantes concordam com o substantivo a que se referem.] 




Erros:  2


 Tais comandos devem ser colocados dentro de um bloco try.  ~  [] 






ETAPA 1.ii) 
===
Verificar Similaridade entre documentos


In [7]:

from similarity import calcula_similaridade
calcula_similaridade(respostas)

levenshtein_distance
 [[nan  0.  0.  0.]
 [nan nan  0.  0.]
 [nan nan nan  0.]
 [nan nan nan nan]]


jaro distance
 [[nan  1.  1.  1.]
 [nan nan  1.  1.]
 [nan nan nan  1.]
 [nan nan nan nan]]


demerau levenshtein distance
 [[nan  0.  0.  0.]
 [nan nan  0.  0.]
 [nan nan nan  0.]
 [nan nan nan nan]]


jaro winkler
 [[nan  1.  1.  1.]
 [nan nan  1.  1.]
 [nan nan nan  1.]
 [nan nan nan nan]]


hamming distance
 [[nan  0.  0.  0.]
 [nan nan  0.  0.]
 [nan nan nan  0.]
 [nan nan nan nan]]


match_rating
 [[nan  1.  1.  1.]
 [nan nan  1.  1.]
 [nan nan nan  1.]
 [nan nan nan nan]]


fuzz.ratio
 [[ nan 100. 100. 100.]
 [ nan  nan 100. 100.]
 [ nan  nan  nan 100.]
 [ nan  nan  nan  nan]]


ETAPA 2
====
Avaliar a similaridade das respostas com base no gabarito. Esta fase é composta pelas seguintes subetapas:
1. Construir um dicionário de sinônimos com base no gabarito, onde o termo do gabarito é a chave.
2. Avaliar termos de cada uma das respostas. Na existência de sinônimos, o termo será substituído pela respectiva  chave do dicionário.

In [107]:
from pysinonimos.sinonimos import Search, historic
gabarito = ['Tais comandos devem ser colocados dentro de um bloco try. Porque no caso de ocorrer um exceção no bloco try, ela será lançado, os demais comandos da bloco serão suspensos, e o controle passará para o primeiro bloco catch que tenha um parâmetro de tipo compatível com a exceção lançada.']
stopwords = nltk.corpus.stopwords.words('portuguese')


gabaritoSemPontuacao=[]
for g in gabarito: 
    gabaritoSemPontuacao.append(re.sub('[,.]', '', g)) #retirar pontuação

words=[]    
for g in gabaritoSemPontuacao:
	words = g.lower().split() #colocar em caixa baixa e dividir sentença em termos


dicionario = dict()
#Cria dicionário com palavras do gabarito qua não estão nas stopwords
for w in words: 
    if w not in stopwords:
        dicionario[w]=[]


#Cada chave do dicionário é atribuída a respectiva lista de sinônimos
for k in iter(dicionario.keys()):
    key = Search(k)
    syn = key.synonyms()
    if (isinstance(syn,list)):
        dicionario[k]=syn
    




False
{'compatível': ['adaptável', 'comportável', 'conciliável', 'harmonizável', 'patível', 'coadunável', 'compossível'], 'passará': [], 'suspensos': ['pêndulos', 'cortados', 'parados', 'detidos', 'estagnados', 'impedidos', 'represados', 'retidos', 'indecisos', 'pendentes', 'pendurados', 'pênseis', 'aéreos', 'atrasados', 'dependurados', 'estacados', 'inseguros', 'interditos', 'interruptos', 'levantados'], 'demais': ['excessivo', 'demasiado', 'exorbitante', 'além da conta', 'além da medida', 'excessivamente', 'demasiadamente', 'intensamente', 'descomedidamente', 'ademais', 'além disso', 'além desse fato', 'outros', 'restantes'], 'lançada': ['apresentada'], 'bloco': ['aglomerado', 'massa', 'caderneta', 'talonário', 'talão', 'caderno', 'união', 'agrupamento', 'ajuntamento', 'batelada', 'bando', 'grupo', 'conjunto', 'quantidade', 'cordão', 'cortejo', 'grêmio', 'sociedade', 'associação', 'coalizão', 'trato', 'aliança', 'acordo', 'quadra', 'quarteirão', 'prédio', 'entrada', 'bloqueio'], 'com